# Imports

In [1]:
pip install transformers

In [11]:
import zipfile
from PIL import Image
import torch
import requests
from io import BytesIO
from transformers import AutoModelForImageClassification, AutoFeatureExtractor

# 파일 로드

**하단의 코드를 실행하기 전에 먼저 디렉토리에 `model_directory.zip` 파일을 업로드해 주세요!**

![step1](https://github.com/oenereaty/DIP2024/blob/main/step1.png?raw=true)

`파일` 아이콘 클릭 → `세션 저장소에 업로드` 아이콘 클릭

![step2](https://github.com/oenereaty/DIP2024/blob/main/step2.png?raw=true)

미리 다운로드 받은 `model_directory.zip` 파일을 업로드 시 사진 같은 모습으로 나타납니다.

업로드 되었다면 하단의 코드를 실행해 주세요!

In [7]:
with zipfile.ZipFile('model_directory.zip', 'r') as z:
    z.extractall('./model_directory')

In [8]:
# 모델 로드
model_directory = "./model_directory"
model = AutoModelForImageClassification.from_pretrained(model_directory)
image_processor = AutoFeatureExtractor.from_pretrained(model_directory)

print("모델과 프로세서가 성공적으로 로드되었습니다!")


모델과 프로세서가 성공적으로 로드되었습니다!


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


# 모델 가동

In [18]:
from PIL import Image
import torch
import requests
from io import BytesIO
from transformers import AutoModelForImageClassification, AutoFeatureExtractor

# 모델 로드
model = AutoModelForImageClassification.from_pretrained("model_directory")
image_processor = AutoFeatureExtractor.from_pretrained("model_directory")

# 라벨과 클래스 이름 및 해결책 매핑
label_map = {
    0: {
        "name": "Apple___Apple_scab",
        "solution": "Apple scab은 주로 과도한 습기로 인해 발생합니다. 감염된 잎과 과일을 제거하고 살균제를 사용하세요."
    },
    1: {
        "name": "Apple___Black_rot",
        "solution": "Black rot은 감염된 가지와 과일을 제거해야 합니다. 또한, 구리 기반 살균제를 사용해 예방하세요."
    },
    2: {
        "name": "Apple___Cedar_apple_rust",
        "solution": "Cedar apple rust는 인근의 주니퍼 나무를 제거하거나, 방제제를 사용해 증상을 완화할 수 있습니다."
    },
    3: {
        "name": "Apple___healthy",
        "solution": "사과 나무는 건강합니다. 추가 조치는 필요하지 않습니다!"
    }
}

# 외부 이미지 URL 처리 함수
def process_image_from_url(image_url):
    try:
        # 이미지 다운로드
        response = requests.get(image_url)
        if response.status_code != 200:
            raise Exception("이미지를 가져올 수 없습니다. URL을 확인하세요.")

        # 이미지 열기
        image = Image.open(BytesIO(response.content))

        # 이미지 채널 변환 (RGB로 강제 변환)
        if image.mode != "RGB":
            image = image.convert("RGB")

        # 모델 입력 준비
        inputs = image_processor(images=image, return_tensors="pt")

        # 모델 예측
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1).item()

        # 라벨 매핑
        prediction_info = label_map.get(predictions, {"name": "Unknown", "solution": "해결책을 찾을 수 없습니다."})

        # 결과 출력
        print(f"예측 결과: {prediction_info['name']}")
        print(f"해결책: {prediction_info['solution']}")
    except Exception as e:
        print(f"이미지 처리 중 오류 발생: {e}")

# 이미지 넣기
image_url = "https://as2.ftcdn.net/v2/jpg/04/98/55/83/1000_F_498558318_eIxJnmctX9YuAPhmsPAvTZHnvZy0qvc8.jpg"
process_image_from_url(image_url)


예측 결과: Apple___healthy
해결책: 사과 나무는 건강합니다. 추가 조치는 필요하지 않습니다!
